In [1]:
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U accelerate
!pip install -U peft
!pip install huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 29.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0


In [2]:
from huggingface_hub import login
login()

In [42]:
base_model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
new_model_id = "bhuvana-ak7/Llama-3.1-8B-sft-lora-mna"

In [43]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from transformers import BitsAndBytesConfig

In [44]:
device_map = "auto"

In [45]:
# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16,
)

In [46]:
base_model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                             quantization_config=quantization_config, 
                                             device_map=device_map)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [47]:
model = PeftModel.from_pretrained(base_model, new_model_id)

In [48]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [36]:
input = '''
instruction: Extract the following from the input M&A news article: announced_date (The date when the M&A deal was announced), acquiring_company(The name of the acquiring company), acquired_company(The name of the acquired company), deal_amount(The deal amount of the merger or acquisition), mna_type(The type of M&A deal: Acquisition or Merger)
input: 
Mars agrees $36bn deal to buy Pringles maker Kellanova
Mars, the chocolate to pet food group, has struck a $36bn (£28bn) deal to buy Kellanova, the maker of Pringles and Pop-Tarts.
The all-cash offer, announced on Wednesday, is the biggest ever acquisition for the privately owned Mars, dwarfing its $23bn takeover of the chewing gum maker Wrigley in 2008.
The deal could attract scrutiny from competition watchdogs as it brings together well-known consumer goods brands including Mars’ Twix, Bounty and Milky Way with Kellanova’s Pringles, Special K cereal and Carr’s water biscuits.
'''

In [49]:
input = '''
instruction: Extract the following from the input M&A news article: announced_date (The date when the M&A deal was announced), acquiring_company(The name of the acquiring company), acquired_company(The name of the acquired company), deal_amount(The deal amount of the merger or acquisition), mna_type(The type of M&A deal: Acquisition or Merger)
input: 
BENGALURU, Aug 28, 2024 - Walt Disney Co (DIS.N), and Reliance Industries (RELI.NS), won approval on Wednesday for an $8.5 billion merger of 
their Indian media assets after assuaging regulatory worries about their grip on broadcasting rights for cricket, India's favourite sport.
India's Competition Commission (CCI) said the deal had been approved subject to some modifications.
The antitrust watchdog had expressed concern that a merged entity would control most cricket rights for TV and streaming in India, 
and could hurt advertisers.
'''

In [50]:
messages = [
    {"role": "user", "content": input},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [51]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [52]:
outputs = model.generate(
    input_ids,
    max_new_tokens=128,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [53]:
response = outputs[0][input_ids.shape[-1]:]

In [54]:
print(tokenizer.decode(response, skip_special_tokens=True))

announced_date': '2024-08-28', acquiring_company': 'Walt Disney Co', acquired_company': 'Reliance Industries', deal_amount': 8.5, mna_type': 'Merger'}
